In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import re
from pprint import pprint
from scipy.sparse.linalg import spsolve

In [2]:
grouped_purchased = pd.read_csv('datasets/online_retail.csv', header=0)
grouped_purchased.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [3]:
grouped_purchased.drop(columns=['InvoiceNo', 'Description', 'InvoiceDate', 'UnitPrice', 'Country'], inplace=True)

In [4]:
grouped_purchased.head()

,StockCode,Quantity,CustomerID
0,85123A,6,17850.0
1,71053,6,17850.0
2,84406B,8,17850.0
3,84029G,6,17850.0
4,84029E,6,17850.0


In [5]:
grouped_purchased = grouped_purchased[['CustomerID', 'StockCode', 'Quantity']]
grouped_purchased.head()

,CustomerID,StockCode,Quantity
0,17850.0,85123A,6
1,17850.0,71053,6
2,17850.0,84406B,8
3,17850.0,84029G,6
4,17850.0,84029E,6


In [6]:
# pattern = r'[A-Z]'
# letters = re.compile(pattern)
# grouped_purchased.StockCode = grouped_purchased.StockCode.apply(lambda x: re.sub(letters, '', x))
grouped_purchased.CustomerID = grouped_purchased.CustomerID.fillna(0)
grouped_purchased.CustomerID = grouped_purchased.CustomerID.astype(int)
grouped_purchased.CustomerID = grouped_purchased.CustomerID.astype(str)

In [7]:
grouped_purchased.head()

,CustomerID,StockCode,Quantity
0,17850,85123A,6
1,17850,71053,6
2,17850,84406B,8
3,17850,84029G,6
4,17850,84029E,6


In [8]:
customers = list(np.sort(grouped_purchased.CustomerID.unique()))
products = list(grouped_purchased.StockCode.unique())
quantity = list(grouped_purchased.Quantity.unique())

print(customers[:10])
print(products[:10])


['0', '12346', '12347', '12348', '12350', '12352', '12353', '12354', '12355', '12356']
['85123A', '71053', '84406B', '84029G', '84029E', '22752', '21730', '22633', '22632', '84879']


In [9]:
cat_dtype_cus = pd.api.types.CategoricalDtype(categories = customers, ordered=True)
cat_dtype_prod = pd.api.types.CategoricalDtype(categories = products, ordered=True)
rows = grouped_purchased.CustomerID.astype(cat_dtype_cus).cat.codes
cols = grouped_purchased.StockCode.astype(cat_dtype_prod).cat.codes
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

ValueError: row, column, and data array must all be the same length

In [ ]:
import pickle
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, auc_score

In [ ]:
item_lookup = pd.read_csv('datasets/online_retail.csv', header=0)
item_lookup.head()

In [ ]:
item_lookup.drop(columns=['InvoiceNo', 'Quantity', 'InvoiceDate', 'UnitPrice', 'Country', 'CustomerID'], inplace=True)
item_lookup.head()

In [ ]:
model = LightFM(loss='warp')
model.fit_partial(product_train, epochs=40, num_threads=4)

with open('save_model', 'wb') as f:
    saved_model = {'model': model}
    pickle.dump(saved_model, f)

train_auc = auc_score(model, product_train).mean()
test_auc = auc_score(model, product_train).mean()

print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc)) 